In [1]:
import pandas as pd
import geopandas as gpd

In [22]:
world = gpd.read_file("ne_110m_admin_0_countries.zip")

In [23]:
world2 = world[["ISO_A3_EH", "ISO_A2_EH"]]

In [24]:
world2.to_excel("Ländercodes.xlsx")

In [19]:
for i in world.columns:
    print(i, "NOR" in world[i].values)
    #print(i, world.loc[121,i])

featurecla False
scalerank False
LABELRANK False
SOVEREIGNT False
SOV_A3 True
ADM0_DIF False
LEVEL False
TYPE False
TLC False
ADMIN False
ADM0_A3 True
GEOU_DIF False
GEOUNIT False
GU_A3 True
SU_DIF False
SUBUNIT False
SU_A3 True
BRK_DIFF False
NAME False
NAME_LONG False
BRK_A3 True
BRK_NAME False
BRK_GROUP False
ABBREV False
POSTAL False
FORMAL_EN False
FORMAL_FR False
NAME_CIAWF False
NOTE_ADM0 False
NOTE_BRK False
NAME_SORT False
NAME_ALT False
MAPCOLOR7 False
MAPCOLOR8 False
MAPCOLOR9 False
MAPCOLOR13 False
POP_EST False
POP_RANK False
POP_YEAR False
GDP_MD False
GDP_YEAR False
ECONOMY False
INCOME_GRP False
FIPS_10 False
ISO_A2 False
ISO_A2_EH False
ISO_A3 False
ISO_A3_EH True
ISO_N3 False
ISO_N3_EH False
UN_A3 False
WB_A2 False
WB_A3 False
WOE_ID False
WOE_ID_EH False
WOE_NOTE False
ADM0_ISO True
ADM0_DIFF False
ADM0_TLC True
ADM0_A3_US True
ADM0_A3_FR True
ADM0_A3_RU True
ADM0_A3_ES True
ADM0_A3_CN True
ADM0_A3_TW True
ADM0_A3_IN True
ADM0_A3_NP True
ADM0_A3_PK True
ADM0_A3_DE Tr

In [2]:
limits = pd.read_excel("pipeline_limits.xlsx")
routes = pd.read_excel("pipeline_routes.xlsx")

In [3]:
limits["from_to"] = limits["From Code"] + "_" + limits["To Code"]
limits.head(2)

,From Country,From Code,To Country,To Code,GWh/d,MWh/y,from_to
0,Algeria,DZ,Italy,IT,448.0,163520000.0,DZ_IT
1,Algeria,DZ,Spain,ES,245.0,89425000.0,DZ_ES


In [4]:
routes.head(2)

,Country,Code,Route
0,Austria,AT,AT->DE
1,Algeria,DZ,DZ->TN->IT->CH->DE


In [5]:
not_in_limits = [x for x in routes["Country"].unique() if x not in limits["From Country"].unique() and x not in limits["To Country"]]
not_in_routes = [x for x in limits["From Country"].unique() if x not in routes["Country"].unique()]
print("Not in Limits:", not_in_limits)
print("Not in Routes:", not_in_routes)

Not in Limits: ['Azerbaijan', 'Iraq', 'Iran', 'Libya', 'Morocco', 'Moldova', 'Russian Federation', 'Syria', 'Turkmenistan', 'Tunisia', 'Turkey']
Not in Routes: ['Bosnia-Herznegovina', 'Germany', 'Latvia', 'Luxembourg', 'Serbia']


In [6]:
#routes["First Transport"] = routes["Route"].apply(lambda x: x[4:6])
#routes_capped = routes[~routes["Country"].isin(not_in_limits)].copy()

routes_capped["Downstream"] = routes_capped["Code"].apply(lambda country: routes_capped.loc[routes_capped["Route"].apply(lambda x: "->"+country in x), "Code"].to_list())
routes_capped["First Transport"] = routes_capped["Route"].apply(lambda x: x[4:6])
routes_capped["from_to"] = routes_capped["Code"] + "_" + routes_capped["First Transport"]

In [7]:
routes["Downstream"] = routes["Code"].apply(lambda country: routes.loc[routes["Route"].apply(lambda x: "->"+country in x), "Code"].to_list())
#routes["First Transport"] = routes["Route"].apply(lambda x: x[4:6])
routes["from_to"] = routes["Code"] + "_" + routes["Route"].apply(lambda x: x[4:6])# routes["First Transport"]

In [8]:
routes.head(2)

,Country,Code,Route,Downstream,from_to
0,Austria,AT,AT->DE,"[AZ, BG, GR, HU, IQ, IR, MD, RO, SK, SY, TM, TR]",AT_DE
1,Algeria,DZ,DZ->TN->IT->CH->DE,[],DZ_TN


In [16]:
test = pd.merge(routes, limits, how = "left", on = "from_to").fillna(0)

In [21]:
test.head(2)

,Country,Code,Route,Downstream,from_to,test,From Country,From Code,To Country,To Code,GWh/d,MWh/y
0,Austria,AT,AT->DE,"[AZ, BG, GR, HU, IQ, IR, MD, RO, SK, SY, TM, TR]",AT_DE,Austriaa,Austria,AT,Germany,DE,150.0,54750000.0
1,Algeria,DZ,DZ->TN->IT->CH->DE,[],DZ_TN,Algeriaa,0,0,0,0,0.0,0.0


In [43]:
test.loc[test["Code"] == "AT", "Downstream"].values[0]
a = 10
for i in test.loc[test["Code"] == "DZ", "Downstream"].values[0]:
    print(a)